
---
---
<strong style="font-size:42pt;">Toivonen</strong> 

---
---

In [224]:
import pandas as pd
import numpy as np
from itertools import combinations, chain
import random

In [163]:
with open('../q3/browsing.txt', 'r') as file:
    data = file.readlines()
    
baskets = []
for line in data:
    baskets.append([product for product in line.split()])
    
items = []
for basket in baskets:
    for item in basket:
        if item not in items:
            items.append(item)

In [185]:
baskets_in_id = []
for basket in baskets:
    basket_items = set()
    for item in basket:
        basket_items.add(items.index(item))
    baskets_in_id.append(list(basket_items))

In [164]:
print(f"Number of items:{len(items)}")
print(f"Number of baskets:{len(baskets)}")

Number of items:12592
Number of baskets:31101


---

- # First pass -> 
---

In [216]:
p = 0.01
support = 100
# support_sample = 0.9 * p * support 
support_sample = 5

In [156]:
def sampling(baskets, probability):
    size = len(baskets)
    return random.sample(baskets, int(probability*size))   

In [255]:
sample = sampling(baskets, p)

In [256]:
sample_baskets = []
for basket in sample:
    basket_items = set()
    for item in basket:
        basket_items.add(items.index(item))
    sample_baskets.append(list(basket_items))

In [257]:
print(sample_baskets)

[[1856, 3680, 323, 2917, 167, 3278, 46, 208, 55, 8023, 57, 986, 311, 5662], [160, 129, 32, 4387, 4004, 779, 108, 525, 528, 894], [1571, 3797, 117, 10647, 10648], [2690, 388, 1381, 10470, 138, 2155, 4400, 3934, 208, 3292, 472, 249, 8794, 444, 1214, 12511], [2688, 225, 4, 9, 3959, 147, 3960, 1075, 790, 535, 2008], [259, 388, 138, 2702, 16, 404, 2070, 288, 38, 41, 1076, 1084, 197, 202, 205, 848, 592, 850, 1875, 1018], [1576, 3050, 236, 283, 123, 667, 990], [1154, 259, 138, 15, 16, 20, 10647, 10783, 288, 38, 1066, 2492, 3647, 199, 10708, 468, 100, 101, 361, 889], [901, 1734, 199, 41, 138, 203, 2153, 2544, 592, 1075, 372, 308, 249, 283, 605, 254, 1823], [1508, 869, 103, 1064, 543, 108, 847, 213, 1145, 4380, 1023], [65, 906, 1261, 2450, 275, 1558, 447], [260, 264, 170, 491, 15, 1108, 10647, 2106, 251, 701], [3936, 1029, 2089, 266, 4626, 1426, 796], [129, 934, 1095, 1375, 841, 845, 975, 4826, 4827, 572, 1631], [64, 1376, 483, 359, 267, 299, 1357, 397, 820, 117, 2936, 251, 2492], [164, 6091, 1

In [204]:
def get_support(itemset):
    return sum(1 for basket in sample_baskets if set(itemset).issubset(set(basket)))

In [326]:
itemsets_count = {}
for basket in sample_baskets:
    for item in basket: 
        if (item,) not in itemsets_count:
            itemsets_count[(item,)] = 0
        itemsets_count[(item,)] += 1
        
frequent_itemsets = {itemset for itemset, count in itemsets_count.items() if count >= support_sample}

In [252]:
def apriori_gen(frequent_itemsets, k):
   
      candidates = [tuple(sorted(set(set1).union(set(set2)))) for set1 in frequent_itemsets for set2 in frequent_itemsets if len(set(set1).union(set(set2))) == k]
      new_frequent_itemsets = {itemset for itemset in set(candidates) if get_support(itemset) >= support_sample}
      return candidates, new_frequent_itemsets

In [248]:
def negative_border_gen(candidates, frequent_itemsets, k):
    negative_border = set()
    for candidate in candidates:
        if candidate not in frequent_itemsets:
            subsets = combinations(candidate, k-1)
            if all(tuple(sorted(subset)) in frequent_itemsets for subset in subsets):
                negative_border.add(candidate)
    return list(negative_border)

In [259]:
k = 2
negative_border = []
while True:
    candidates, new_frequent_itemsets = apriori_gen(frequent_itemsets, k)
    negative_border.extend(negative_border_gen(candidates, new_frequent_itemsets, k))
    if not new_frequent_itemsets:
        break
    frequent_itemsets.update(new_frequent_itemsets)
    print(k)
    k += 1

2
3


---

- # Second pass -> 
---

In [201]:
def get_full_support(itemset):
    return sum(1 for basket in baskets_in_id if set(itemset).issubset(set(basket)))

In [277]:
valid_itemsets = {itemset: get_full_support(itemset) for itemset in frequent_itemsets if get_full_support(itemset) >= support}

In [278]:
if not any(get_full_support(itemset) >= support for itemset in negative_border):
    status = True

In [279]:
print(f"Status of algorithm: {status}")

Status of algorithm: True


In [280]:
print(f"Frequents itemsets: {valid_itemsets}")

Frequents itemsets: {(1,): 3875, (1, 15): 580, (1, 136): 146, (1, 138): 1592, (1, 199): 697, (1, 288): 547, (1, 349): 173, (1, 574): 186, (1, 1023): 115, (4,): 906, (9,): 2292, (9, 138): 650, (9, 199): 346, (10,): 380, (15,): 3602, (15, 16): 304, (15, 78): 380, (15, 117): 305, (15, 138): 1139, (15, 138, 199): 315, (15, 164): 318, (15, 199): 882, (15, 214): 320, (15, 231): 227, (15, 236): 172, (15, 251): 486, (15, 252): 313, (15, 259): 631, (15, 283): 562, (15, 288): 712, (15, 299): 427, (15, 349): 344, (15, 538): 225, (15, 855): 137, (15, 914): 389, (15, 2492): 202, (16,): 1219, (16, 100): 227, (16, 138): 297, (16, 288): 215, (17,): 2083, (17, 538): 155, (18,): 235, (20,): 655, (20, 138): 155, (22,): 599, (23,): 1531, (27,): 401, (31,): 1311, (32,): 349, (34,): 1627, (36,): 255, (37,): 469, (38,): 1697, (38, 138): 279, (39,): 664, (41,): 1489, (41, 138): 479, (41, 251): 204, (45,): 695, (46,): 832, (53,): 1094, (57,): 825, (64,): 811, (65,): 703, (66,): 915, (71,): 1702, (72,): 664, (7

### Compute Confidence for rules

In [322]:
frequent_doubletons = []
frequent_triplets = []
for itemset in valid_itemsets:
    if len(itemset) == 2:
        frequent_doubletons.append(itemset)
    elif len(itemset) == 3:
        frequent_triplets.append(itemset)


In [323]:
for i in frequent_triplets:
    print(i)

(15, 138, 199)
(138, 199, 283)
(138, 283, 288)
(138, 283, 901)
(199, 203, 283)
(199, 283, 288)
(199, 283, 290)
(199, 283, 901)


In [327]:
ass_double_rules = {}
for i in frequent_doubletons:
    
    ass_double_rules[(i[0], i[1])] = valid_itemsets[i] / get_full_support((i[0],))
    ass_double_rules[(i[1], i[0])] = valid_itemsets[i] / get_full_support((i[1],))

sorted_rules = sorted(ass_double_rules.items(), key=lambda x: x[1], reverse=True)
top_doubleton_rules = [rule for rule in sorted_rules if rule[1]>0.5]
print(f"Top rules with confidence above 0.5:\n {top_doubleton_rules}")
print(f"\n Number of top rules: {len(top_doubleton_rules)}")

Top rules with confidence above 0.5:
 [((203, 199), 0.999176276771005), ((290, 199), 0.983510011778563), ((1327, 138), 0.7326649958228906), ((235, 231), 0.6016483516483516), ((2492, 138), 0.5976714100905562), ((260, 251), 0.5511627906976744), ((1087, 138), 0.5394871794871795), ((901, 199), 0.530562347188264), ((901, 283), 0.5012224938875306)]

 Number of top rules: 9


In [320]:
ass_triplet_rules = {}
for i in frequent_triplets:
    ass_triplet_rules[(i[0], i[1], i[2])] = valid_itemsets[i] / get_full_support((i[0], i[1]))
    ass_triplet_rules[(i[0], i[2], i[1])] = valid_itemsets[i] / get_full_support((i[0], i[2]))
    ass_triplet_rules[(i[1], i[2], i[0])] = valid_itemsets[i] / get_full_support((i[1], i[2]))

sorted_triplet_rules = sorted(ass_triplet_rules.items(), key=lambda x: x[1], reverse=True)
top_triplet_rules = [rule for rule in sorted_triplet_rules if rule[1]>0.5]
print(f"Top rules with confidence above 0.5:\n {top_triplet_rules}")
print(f"\n Number of top rules: {len(top_triplet_rules)}")

Top rules with confidence above 0.5:
 [((203, 283, 199), 1.0), ((283, 290, 199), 0.9744318181818182), ((283, 901, 199), 0.5658536585365853), ((138, 901, 283), 0.5563636363636364), ((199, 901, 283), 0.5345622119815668), ((138, 283, 199), 0.5157096424702059)]

 Number of top rules: 6
